# Scraping FBref for data about Eliteserien results for the seasons 2021, 2022 and 2023.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import random

we are going to scrape the 2021, 2022 and 2023 seasons, starting from the 2023 season. 
all_matches is an empty list that will in the end contain several dataframes, and each dataframe is going to contain the matchlog for one team in onea season. 
standings_url is where the scraper will start, which is the 2023 season of Eliteserien. 

In [2]:
years = list(range(2023, 2020, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/28/2023/2023-Eliteserien-Stats"

In this for loop we go through every year given in the years variable. 
First we assign the starting url to a data variable with requests, before using BeautifulSoup to parse said html. We are looking for the standings table, as this includes all the links to the different team in the league that year, so we assign this to the standings_table variable.

The next step is to find all the team links and get the href, and filter the links to only have the squad links. We then turn the relative links we gathered and turn them into absolute links that we save in the variable team_urls.

We also want to grab the relative link for the previous season and make that an absolute link to gather data from previous seasons.

In the nested for loop we loop through each team url and scrape the match log for each individual team. We also want to gather shooting stats for each match, so we get the link for this as well. The next step is to try to merge the matches and shooting data by the date the match was played on.

The final step is to crate bew columns in team_data and append this to the all_matches we created earlier. This loop will go trough every match for every team for every year we have selected. 

There are a lot of time.sleep lines, as the website would block the requests, the code takes roughly 12 minutes to run for the given seasons. 

In [5]:
for year in years:
    data = requests.get(standings_url)
    time.sleep(random.randint(4, 7))
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        time.sleep(random.randint(4, 7))
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        time.sleep(random.randint(4, 7))
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

C:\Users\Bakke\AppData\Local\Temp\ipykernel_113368\242995783.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Bakke\AppData\Local\Temp\ipykernel_113368\242995783.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Bakke\AppData\Local\Temp\ipykernel_113368\242995783.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Bakke\AppData\Local\Temp\ipykernel_113368\242995783.py:24: FutureWarning: Passing literal 

The final steps in the scraping part of this project is to combine all the dataframes, and lowercase all the columns before converting the dataframe to a csv file. 

In [7]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("matches.csv")